# Literary Knowledge Network – Cluster Explorer

Summarize community clusters with representative works, keywords, tags, and descriptions to support manual genre labeling.

In [1]:
from pathlib import Path
from textwrap import shorten
from typing import Iterable, List

import numpy as np
import pandas as pd

In [2]:
REPO_ROOT = Path('..').resolve()
DATA_DIR = REPO_ROOT / 'data'

paths = {
    'work_enriched': REPO_ROOT / 'data/processed/enriched/work_final.parquet',
    'work_communities': REPO_ROOT / 'data/output/communities/work_communities.parquet',
}

work_df = pd.read_parquet(paths['work_enriched'])
community_df = pd.read_parquet(paths['work_communities'])
community_df = community_df.sort_values(['cluster_resolution', 'cluster_id'])
print(f"Loaded {len(work_df)} enriched works and {len(community_df)} community assignments.")

Loaded 3581 enriched works and 2470 community assignments.


In [3]:
def flatten_terms(value) -> List[str]:
    if value is None or (isinstance(value, float) and np.isnan(value)):
        return []
    if isinstance(value, str):
        trimmed = value.strip()
        return [trimmed] if trimmed else []
    if isinstance(value, (list, tuple, set)):
        terms: List[str] = []
        for item in value:
            terms.extend(flatten_terms(item))
        return terms
    if hasattr(value, 'tolist'):
        try:
            items = value.tolist()  # type: ignore[assignment]
        except Exception:
            items = list(value)
        return flatten_terms(items)
    return [str(value)]


def top_terms(series: pd.Series, limit: int = 6) -> List[str]:
    exploded = series.dropna().apply(flatten_terms).explode()
    if exploded.empty:
        return []
    counts = exploded.value_counts()
    return [f"{term} ({int(count)})" for term, count in counts.head(limit).items()]


def format_titles(titles: Iterable[str], limit: int = 5) -> str:
    series = pd.Series(list(titles), dtype="object")
    unique_titles = []
    for title in series.dropna().unique():
        if isinstance(title, str):
            trimmed = title.strip()
            if trimmed:
                unique_titles.append(trimmed)
    return '\n'.join(unique_titles[:limit])

In [4]:
CLUSTER_RESOLUTION = 1.0
CLUSTER_MIN_SIZE = 5
TOP_TITLES = 5
TOP_TERMS = 6

merged_df = community_df[community_df['cluster_resolution'] == CLUSTER_RESOLUTION].merge(
    work_df,
    on='work_id',
    how='left',
)

summary_rows = []
for cluster_id, group in merged_df.groupby('cluster_id'):
    cluster_size = int(group['cluster_size'].iloc[0]) if 'cluster_size' in group else len(group)
    if cluster_size < CLUSTER_MIN_SIZE:
        continue
    titles = group['original_title'].dropna()
    keywords = top_terms(group['keywords'], limit=TOP_TERMS)
    tags = top_terms(group['tags'], limit=TOP_TERMS)
    series_ids = top_terms(group['best_book_series_ids'], limit=TOP_TERMS)

    summary_rows.append(
        {
            'cluster_id': int(cluster_id),
            'cluster_size': cluster_size,
            'representative_titles': format_titles(titles, limit=TOP_TITLES),
            'top_keywords': ' '.join(keywords),
            'top_tags': ' '.join(tags),
            'top_series_ids': ' '.join(series_ids),
        }
    )

cluster_summary_df = pd.DataFrame(summary_rows).sort_values('cluster_size', ascending=False).reset_index(drop=True)
cluster_summary_df

,cluster_id,cluster_size,representative_titles,top_keywords,top_tags,top_series_ids
0,0,214,Justice Served\nJustice in the Shadows\nDying ...,they (5) murder (5) fbi (4) civil (3) agent (3...,['ansari' 'body' 'dean' 'donald' 'her' 'him' '...,[] (23) ['415778'] (1) ['1007364'] (1) ['41577...
1,1,189,Medusa's Gaze and Vampire's Bite: The Science ...,vampire (23) vampires (4) blood (3) queen (3) ...,,[] (15) ['179149'] (1) ['179145'] (1) ['156478...
2,2,149,To Pleasure a Duke\nWhat the Duke Wants\nWhen ...,sophie (6) lady (5) man (4) husband (3) weddin...,,[] (19) ['267068'] (1) ['164328'] (1) ['162852...
3,3,134,The Haunting\nSignificance\nSix Myths of Our T...,cassie (5) maggie (4) brother (3) love (3) fam...,,[] (24) ['274177'] (1) ['390425'] (1) ['516402...
4,4,115,"Authors of the Impossible\nFables, Volume 4: M...",horror (4) stories (3) ghost (3) fiction (3) b...,,[] (20) ['214342' '538267'] (1) ['839441'] (1)...
...,...,...,...,...,...,...
81,81,5,Winnie-the-Pooh and the Blustery Day\nPooh and...,cleverly (1) psychologist (1) leading them (1)...,,[] (2) ['317652'] (1)
82,82,5,The Sheikh's Wedding Contract,zayed (1) sheikh (1) allegiance (1) begs (1) a...,,['757215'] (1)
83,83,5,Spectrums of Possibility: When Psychology Meet...,tarot (2) clinical (1) practicing (1) card (1)...,,[] (2)
84,84,5,,,,


### Inspect A Single Cluster

Use `inspect_cluster` to view the works in a specific cluster with truncated descriptions for quick review.

In [5]:
def truncate_description(text: str, width: int = 300) -> str:
    if not text or not isinstance(text, str):
        return ''
    return shorten(' '.join(text.split()), width=width, placeholder='…')


def inspect_cluster(cluster_id: int, max_rows: int = 25) -> pd.DataFrame:
    subset = merged_df[merged_df['cluster_id'] == cluster_id].copy()
    if subset.empty:
        raise ValueError(f'Cluster {cluster_id} not found at resolution {CLUSTER_RESOLUTION}.')

    subset['keywords_preview'] = subset['keywords'].apply(lambda x: ', '.join(flatten_terms(x)[:8]))
    subset['tags_preview'] = subset['tags'].apply(lambda x: ', '.join(flatten_terms(x)[:8]))
    subset['series_preview'] = subset['best_book_series_ids'].apply(lambda x: ', '.join(flatten_terms(x)[:5]))
    subset['description_preview'] = subset['best_book_description'].apply(truncate_description)

    columns = [
        'work_id',
        'original_title',
        'cluster_size',
        'keywords_preview',
        'tags_preview',
        'series_preview',
        'description_preview',
    ]
    return subset[columns].sort_values('original_title').head(max_rows)

# Example usage: display(inspect_cluster(5))

In [6]:
display(inspect_cluster(0))

,work_id,original_title,cluster_size,keywords_preview,tags_preview,series_preview,description_preview
70,25270797,A Born Victim,214,"taylor, lucy, against, case, women, kick, coul...",,[],"'A Born Victim' is the story of Gill Brogan, a..."
123,46097286,Babytrick,214,"final, black, coleman, ever imagine, read stan...",,['814193'],Camie at 14 is a wild Detroit street girl on t...
151,6728538,Bedded by Blackmail,214,"tristan, ella, turned, sworn off, guarantee, t...",,[],"Burned by a greedy heiress, Tristan Barkley ha..."
107,428278,Buried in Time (Nancy Drew and the Hardy Boys:...,214,"carolyn, oklahoma, nancy drew, unmarked, paths...",,['149126'],/Carolyn Keene /Carolyn Keene Nancy Drew inves...
60,24271812,Camille,214,"artist, parisian, forced confront, aspirations...",,[],Camille is a psychological novel of growth. Th...
6,6268294,Compulsion,214,"killers, serial killers, stone, california, se...",,['492378'],When Serial Killers Terrorize a California Bea...
83,26341000,Cop Town,214,"slaughter, karin slaughter, karin, cop, first ...",,[],"NEW YORK TIMES BESTSELLER Karin Slaughter, aut..."
12,11943863,Dangerous to Know (Lady Emily #5),214,"emily, normandy, victims, young woman, lady, k...",,['176738'],"Set in the lush countryside of Normandy, Franc..."
32,150827,Dark Dreams: A Legendary FBI Profiler Examines...,214,"hazelwood, roy, sexual, innovative, criminals,...",,[],Profiler Roy Hazelwood is the world's leading ...
73,25640491,"Dark Hunter (Dark Breed Enforcers, #2.5)",214,"mari, breeds, plot, side, preservation, plot a...",,['553954'],Vampire Mari Ankamunan struggles to pick up th...


In [7]:
display(inspect_cluster(1))

,work_id,original_title,cluster_size,keywords_preview,tags_preview,series_preview,description_preview
302,25648439,A Field Guide to Vampires: (and Other Creature...,189,"steve, vampires, creatures, blood, satan, all ...",,[],"Not everybody likes the vampires, werewolves, ..."
344,5542535,A Summer Sentence,189,,,['874203'],
246,1612164,An Unlikely Alliance (Zebra Regency Romance),189,"magda, predictions, gypsy, expose, lord, tarot...",,[],Gypsy fortune teller Mademoiselle Magda has ta...
224,525790,"Bad Blood (Crimson Moon, #1)",189,"sasha, government, trained, team, bump night, ...",,['179145'],Sasha Trudeau knows all about working beneath ...
329,43004856,Blood & Soul,189,,,[],
250,16410551,Blood Born,189,"kalan, bred, council, blade, territory, human,...",,['275905'],Her lineage is both a blessing and a curse... ...
240,15130163,Blood Eternal,189,"elizabeth, vampire, human population, unites, ...",,['224892'],"Despite the growing passion that unites them, ..."
331,54098634,Blood Moon,189,,,['1007378'],
249,43081721,Blood in Fire,189,"aidan, darkness, heather, ireland, wish, can, ...",,['727942'],This is an alternate cover edition for  Aidan...
298,24167202,Blood of Zee: Unfinished Business,189,"people around, philadelphia, mischievous, unfo...",,[],A troubled Philadelphia girl finds that the pe...


In [8]:
display(inspect_cluster(2))

,work_id,original_title,cluster_size,keywords_preview,tags_preview,series_preview,description_preview
446,44718076,"A Scorching Dilemma (The Rue Alliance, #2)",149,"daniel, faith, but, fire, choice, man, family, so",,['748248'],Inspired by Cinderella HE'S NOT EXACTLY PRINCE...
455,21848486,Allerednic: A Regency Cinderella Tale--In Reverse,149,"most eligible, misses, ever loved, mart, propo...",,['506376'],"Lady Charlotte Eversley, heir of the Earl of E..."
453,47205830,Always With Love,149,"billy, sophie, love story, their differences, ...",,['895170'],The next much-anticipated romantic chapter in ...
498,50609392,Bender,149,"mace, dex, pain, sexual, sees, cover artist, s...",,[],"At nineteen, college freshman Mace Danner work..."
459,23756518,"Bewitch (Vampire Erotic Theatre Romance, #5)",149,"elissa, payne, surrendering, fulfilling, promi...",,['460261'],A vampire with a past stained with blood and a...
434,15343817,"Claimed (Brides of the Kindred, #1)",149,"kindred, baird, liv, scourge, olivia, claiming...",,['244049'],Olivia Waterhouse has just graduated from nurs...
502,524854,Claiming the Courtesan,149,"verity, courtesan, comes close, way possible, ...",,[],HE WOULD MARRY HER AND POSSESS HER IN EVERY WA...
437,20385623,Clifftop Fantasies: BDSM Menage Fantasies,149,"allie, brad, karl, father, deal, sexy, rather ...",,['511364'],"When Allie Walker inherits her father's huge, ..."
500,52115904,Dangerously Sweet,149,"dex, amelia, barefoot, bay, barton, recipe, li...",,['950100'],"Blake ""Dex"" Dexter has been undercover so long..."
445,2017097,Duke Most Wanted,149,"sophie, graham, seal, content, part, no, scoun...",,['160240'],Sophie Blake's grandfather willed his fortune ...


In [9]:
display(inspect_cluster(3))

,work_id,original_title,cluster_size,keywords_preview,tags_preview,series_preview,description_preview
648,870116,A Dry Spell,134,"nina, repercussions, seems, jane, now, coming ...",,[],In 1976 four students took a trip to the deser...
558,11123843,"A Lesson in Forgiveness (Lesson Series, #2)",134,"ginny, marry, playing, all knows, work new, ow...",,['390425'],"Ginny figured being a 21st century woman, stuc..."
646,7226209,A Life Revealed,134,"chloe, tragedy, life while, find truth, order ...",,[],Tragedy can strike in a split second. But what...
584,18415657,A Promise for Miriam,134,,,['391119'],
567,22016269,A Treacherous Likeness,134,"shelley, poet, charles, wife mary, frankenstei...",,['516402'],A mystery that explores the dark lives and une...
640,6130425,A Widow's Hope,134,"hannah, deacon, sheep, seth, ohio, interest, l...",,['188012'],A bright new voice shares a tender love story ...
572,26162458,"Baby, Be Mine",134,"gage, katy, can remember, remember, long, baby...",,['559444' '674501'],From New York Times andUSA Today bestselling a...
619,27361289,Beautiful Dreams,134,"kyle, lily, broken heart, mature content, fall...",,['585590'],"Kyle has mended Lily's broken heart, and as th..."
589,24566785,Before You Go,134,"tabby, noah, can, adler, take action, take bac...",,['516403'],Paralyzed by the past and terrified of the fut...
645,6723844,Beyond Innocence,134,"edward, florence, makes, family, brother, sinf...",,['181900'],Emma Holly turns up the heat in this sinfully ...


In [10]:
display(inspect_cluster(4))

,work_id,original_title,cluster_size,keywords_preview,tags_preview,series_preview,description_preview
760,50533817,A Masquerade of Muertos,115,"wisteria, ellen, halloween, spirit, fits, goth...",,['953906'],"Goths reenacting Poe's ""The Masque of the Red ..."
701,16905470,A Professional Psychic Reading as Alternative ...,115,"psychic, reading, reader, learn, information, ...",,[],MY PSYCHIC READER SAID...Ears perk up when a s...
759,45340893,Aickman's Heirs,115,"aickman, writers, robert, horror, boom, last c...",,[],"""Robert Aickman was a master of what he called..."
686,10711804,Authors of the Impossible,115,"paranormal, thru, history, sacred, religion, s...",,[],Most scholars dismiss research into the parano...
726,57406849,Beechwood Harbor Ghost Mysteries,115,"ghost, ghosts, being, don, kristen, thing, had...",,['1134100'],The first rule of being a ghost whisperer: Don...
693,140077,Best Ghost Stories of J.S. Le Fanu,115,"ghost, haunted, classic, ghost stories, haunte...",,[],"Here are 16 classic ghost stories: ""Carmilla"" ..."
743,3219843,Cthulhu's Dark Cults,115,"call, fiction, lovecraft, real, deranged, expa...",,[],"Chaosium's ""Call of Cthulhu"" is an endless sou..."
740,24981292,Darkling Light,115,"short, collection, workshop, therapist, killer...",,[],"""Darkling Light... And Other Short Stories"" is..."
692,42578981,"Fables, Volume 22: Farewell",115,"fables, 150, illustrated, mark, final, neal, b...",,['677431' '724349'],It's the final trade paperback volume of Fable...
689,1330999,"Fables, Volume 4: March of the Wooden Soldiers",115,"everyone, bill willingham, willingham, when li...",,['214342' '538267'],Bill Willingham's runaway hit series continues...


In [11]:
display(inspect_cluster(5))

,work_id,original_title,cluster_size,keywords_preview,tags_preview,series_preview,description_preview
850,48181302,"""The Girl with No Name,"" included in the KICKI...",115,"novelette, urban fantasy, urban, fantasy, chri...",,['1035394'],Chris Marie Green brings you an urban fantasy ...
853,953211,A Ghost of a Chance (Haunting Hearts),115,"isadora, oh, law, son, sparked, everything cou...",,[],She glared at him as he sat behind the desk fl...
882,51589454,All Grown Up,115,"andrea, entirely, friend, sister, vignettes, s...",,[],From the New York Timesbest-selling author of ...
810,15689,Bi Fang kara 12,115,"prophecy, their, evil, magic, magic they, mixi...",,['305127'],After a terrorist bomb catapults her into an a...
819,6276917,Captive Spirit: A Novel of the Dark Crescent S...,115,"duncan, bela, worse, soul, satanic, has two, w...",,['169506'],"Trapped by temptation, bound by desire Saving ..."
831,46833607,Chaos Rises,115,"veil, chaos, my brother, news, me my, fall, so...",,['826082'],"'When the veil fell, any half bloods not stron..."
849,41051303,"Conjuring Darkness (Darkness Series, #1)",115,"lexi, ancient, letter, sent, learns, sister, t...",,['629364'],25 year old Lexi Salenko is an independent and...
837,24978467,Curse of the Dream Witch,115,"olivia, witch, dream, dread, leo, uncle, kingd...",,[],It is the age of the Great Dread. The Dream Wi...
877,47298908,Dead Reckoning,115,"neal, byron, maddox, former navy, navy seal, t...",,['1060491'],Egan Maddox isn't sure he's the right man for ...
830,2174932,Death's Shadow,115,"no longer, demons, longer, face, but wasn, gre...",,['161062'],The apocalypse came and the world burned. But ...


In [12]:
display(inspect_cluster(6))

,work_id,original_title,cluster_size,keywords_preview,tags_preview,series_preview,description_preview
960,26212562,3 Book Bundle,108,"allie, flames, fun, love, inches closer, fulle...",,['574211'],"3 BOOK BUNDLE!! ""Her Last Love Affair Series"" ..."
927,42908323,A Handful Of English Classic,108,"their children, charlotte, husband, wellington...",['but' 'charlotte' 'english' 'her' 'husband' '...,[],The novel A Handful of English Classic by Miry...
920,26314782,After We Fell,108,"hardin, tessa, biggest, life, able, someone, f...",,['697548'],Book 3 of the After series--newly revised and ...
931,24248301,Antipodean Collection,108,"short, collection, strange case, suits, advanc...",['advanced' 'androids' 'authors-books-reviewed...,[],A collection of six short-short science fictio...
972,88291,Aphrodite's Daughters : Women's Sexual Stories...,108,"aphrodite, daughters, spiritual, path, overvie...",,[],"Since its publication, this book has become an..."
948,21459357,"Because You Tempt Me (Because You Are Mine, #1.1)",108,"ian, francesca, first, bewildering, cocktail, ...",,['399736'],"It starts with that first look, when you know ..."
929,4646319,Best of Women's Short Stories,108,"charlotte, mary, stories, lover, sometimes, ed...",,[],This rich collection of stories explores the e...
975,403943,Bride of Dark and Stormy (Bulwer-Lytton Contest),108,,,['198411'],
985,46083546,Colors in the Dark,108,"expensive, puzzle, those, love, waiting, there...",,[],An alternate cover for this ASIN can be found ...
978,41243870,"Coulda, Woulda, Shoulda (Once Upon a Theme #1)",108,"elle, endings, don, ll, valerie, beware, find ...",,[],Are you an expert at the game of love? Find ou...


In [13]:
display(inspect_cluster(7))

,work_id,original_title,cluster_size,keywords_preview,tags_preview,series_preview,description_preview
1089,45616222,"A Bear In Hiding (The Marked, #2)",107,"brian, emma, bear, gentle, shifter, encounter,...",,['814062'],Brian Mitchell has been living a double life. ...
1065,21833430,Acquired in Time (Marked 2),107,"menage, kane, their, pack, mate, bring, enough...",,['429572'],[Menage Amour ManLove: Erotic Alternative Para...
1029,18567907,After Dark,107,"mac, choice but, no choice, hours, humans, cho...",,['333718'],What eighteen year old Mac Thorne doesn't know...
1084,41856494,Alpha Heart,107,"emma, travis, wolf, inexplicable, bitten, more...",,[],"Emma never wanted to be a wolf, not since she ..."
1028,13578604,Born at Midnight,107,"kylie, shadow falls, derek, shadow, falls, luc...",,['206026'],Don't miss this spectacular new series that wi...
1036,24255026,Creature Feature,107,"frank, cael, caleb, andrew, can deal, rides, b...",,['682027' '980486'],Diagnosis Wolf by Poppy Dennison Thanks to his...
1056,2362332,Cry Wolf,107,"anna, charles, omega, werewolves, pack, wolf, ...",,['161923' '599755'],"Anna never knew werewolves existed, until the ..."
1030,15880466,Dark of the Moon,107,"guardians, connor, dark, intense, never, ll, t...",,['247560'],"I've loved him forever, but he can never be mi..."
1049,19037981,Defensive Guard,107,"evan, tyler, grizzlies, pines, twin, alpha, er...",,['346927'],[Siren Everlasting Classic ManLove: Erotic Alt...
1044,19128607,"Dire Desires (Eternal Wolf Clan, #3)",107,"dire, wolf, wolves, pack, now only, purgatory,...",,['413650'],"Feared by humans, envied by werewolves, the Di..."


In [14]:
display(inspect_cluster(8))

,work_id,original_title,cluster_size,keywords_preview,tags_preview,series_preview,description_preview
1202,47334525,All These Hidden Scars,102,"sullivan, carter, random, caught middle, sulli...",,['860812'],"When I leave, I never return the same Pretendi..."
1133,13424947,Ashlynn's Dreams,102,"jillian, danielle, held, facility, kidnapped, ...",,['701922'],"Before she was kidnapped, Jillian Marie Antel ..."
1147,21765769,"BTW, I Love You",102,"jessie, same, right, mr, bed, look, ruby, infu...",,['444905'],"Surf, Sea And A Sexy Stranger Millionaire Ryan..."
1151,316485,"Baby, I'm Yours",102,"expected, sitting, no matter, got, matter, jus...",,[],A Ride On the Wild Side The last place Catheri...
1177,25468917,Bared,102,"aidan, cora, club, submissive, erotic, conceal...",,['518574'],In a novel that's perfect for fans of Shayla B...
1193,6893679,Bittersweet Homecoming,102,"gray, andi, perfect, not love, not perfect, on...",,['172411'],Book 2 in the Lawyers in Love series One hot w...
1199,47248387,Dirty Pleasures (The Dirty Billionaire Trilogy...,102,"dirty, my, pleasures, billionaire, trilogy, gi...",,['836983'],I did it. I married a billionaire. My reasons ...
1138,18061257,Free Falling,102,"agency, normal, letting go, seen all, out, rea...",,[],Special Agent Rainie McClain wants out. Out of...
1137,21842224,Hot Summer Nights,102,"luke, jane, fun, gets, carly, divorcee, jane h...",,['522439' '1019220' '1019221'],Passions ignite in these all-new stories from ...
1150,43136320,Hot for You: ...games for two,102,"wellington, melanie, two people, hot, pilot, w...",,['762040'],An alternate cover edition can be found . She...


In [15]:
display(inspect_cluster(9))

,work_id,original_title,cluster_size,keywords_preview,tags_preview,series_preview,description_preview
1252,7790670,A Beautiful Dark,92,"skye, devin, asher, ever imagine, threatens sh...",,['301887'],"On the night of Skye's seventeenth birthday, s..."
1234,13018221,"Absolution (Heavenly, #3)",92,"zoe, matthias, hell, soul, will use, albert, e...",,['177325'],Zoe's secret is out. And the powers of Hell ar...
1249,25442975,All Fall Down,92,"grace, embassy row, embassy, can control, row,...",,['516827'],A new series of global proportions -- from mas...
1265,19075528,Angel in Chains,92,"az, azrael, angel death, could use, dark power...",,['353630'],"Mortal Sin As a fallen Angel of Death, Azrael ..."
1286,289487,Angels on Fire,92,"angel, lucy, when found, decide whether, lure,...",,[],Struggling artist Lucy Blackford though she'd ...
1275,21738025,"Conquering the Dark Axe (The Northern Knights,...",92,"alexa, strong, sister, betrothed, hellion, mos...",,['482489'],A match made in hell... Stubborn and Defiant H...
1282,26724756,Cusala,92,"believe, really, asin can, christians, cover a...",,[],An alternate cover for this ASIN can be found ...
1291,41391710,Dragon Light,92,"elven, alliances, dragons, dawn, queen, final,...",,['633711'],"Dawn clashes with the dark, until moonlight fa..."
1300,7130900,"Edge Of Midnight (McClouds & Friends, #4)",92,"liv, sean, wanting, close, same, shannon, arso...",,['173048'],Shannon McKenna creates characters readers nev...
1245,24670263,Escaping Reality,92,"infinite, liam, amy, wants, shouldn, what, wha...",,['487167'],Alternate cover image for  Infinite possibili...


In [16]:
display(inspect_cluster(10))

,work_id,original_title,cluster_size,keywords_preview,tags_preview,series_preview,description_preview
1356,6247615,A Cowboy Christmas,74,"cowboy, christmas, knight, one only, their dau...",,[],MR. DECEMBER Leander Knight: A rugged cowboy f...
1338,25467144,A Holiday Christmas,74,"christmas, annual christmas, annual, holiday, ...",,['529175'],There's magic in the air this Christmas Season...
1339,18313014,"A Perfect Time for Pandas (Magic Tree House, #48)",74,"tree house, jack annie, magic tree, annie, tre...",,['320910' '369223' '913792' '1076598'],Jack and Annie are ready for their next advent...
1333,18462439,An Epiphany Gift for Robin,74,"arthur, christmas, baron, gift, log, no better...",,[],UPDATED WITH AN EXTENDED ENDING 1/3/2015 Arthu...
1325,1030790,An Undeniable Rogue,74,"shameless, gideon, sabrina, arranged, rogue, h...",,['154370'],An arranged marriage leads to unlikely passion...
1332,975952,Babar et le pere Noel,74,"christmas, father christmas, father, named, br...",,['445336'],Babar returns in this unusual and heartwarming...
1350,22276289,Christmas Eve,74,"christmas eve, eve, christmas, heart warming, ...",,[],'Christmas Eve' is a simple Winter Ghost Story...
1329,13117562,"Darkest Powers Trilogy (Darkest Powers, #1-3)",74,"darkest powers, edison, edison group, group, c...",,['1045528'],"The Summoning The Darkest Powers Series, Book ..."
1334,21943054,Finding Christmas - Santa's Tale,74,"santa, treat, holiday, christmas, finding, bad...",,[],Even Santa has a bad day! Finding Christmas - ...
1364,3247053,Let It Snow: Three Holiday Romances,74,"snow, charming, would, love, back true, beauti...",,[],Alternate cover edition for  The New York Tim...


In [17]:
display(inspect_cluster(11))

,work_id,original_title,cluster_size,keywords_preview,tags_preview,series_preview,description_preview
1416,7194000,A History of Christianity: The First Three Tho...,60,"faith, book, macculloch, believers, christiani...",,[],The author of The Reformationreturns with the ...
1431,46883227,Genesis,60,,,['970751'],
1404,5033272,"Gospel of Mark, The (Catholic Commentary on Sa...",60,"catholic, commentaries, scripture, meeting, st...",,['507094'],There is an increasing hunger among Catholics ...
1402,1154962,Hebrews 9-13,60,"biblical, theological, scholarship, framework,...",,['570142'],The Word Biblical Commentary delivers the best...
1407,1972784,In the Beginning: The Story of the King James ...,60,"bible, translation, english, james, king, earl...",,[],"In the sixteenth century, to attempt to transl..."
1419,25337789,Jesus: A Pilgrimage,60,"jesus, martin, gospels, life jesus, scripture,...",,[],"James Martin, SJ, gifted storyteller, editor a..."
1429,40122804,No Ocean too Deep,60,"republic, human, sea, superiors, tenuous, hims...",,[],"To buck the brutal rule of the Republic, Amirz..."
1409,159302,Old Testament Exegesis: A Handbook for Student...,60,"step, stuart, substantial, new generation, rel...",,[],This substantial revision of a highly successf...
1427,370960,Parmenides of Elea: Fragments: A text and tran...,60,"fragments, translation, poem, also includes, t...",,[],David Gallop provides a Greek text and a new f...
1436,934022,Preaching,60,"preaching, clergy, seminary, preparation, fund...",,[],Painstakingly prepared for seminary students a...


In [18]:
display(inspect_cluster(12))

,work_id,original_title,cluster_size,keywords_preview,tags_preview,series_preview,description_preview
1461,151310,12 Fabulously Funny Fairy Tale Plays,58,"fairy, tales, read aloud, these hilarious, gra...",,[],Students will have oodles of fun building esse...
1469,19055732,A Gentleman's Luck,58,"luck, uncle, rotten, plans, lisa, has, william...",,[],William has suffered through the Civil war and...
1471,42791567,"Angel Unchained, Book Two of The Aliis Mundi S...",58,"princess, life, has, control, djinn, among tho...",,[],"In her life, Azriella Carnadine, the Aliisian ..."
1488,48682039,Before She Ignites,58,"mira, fallen isles, isles, fallen, treaty, bet...",,['867267'],Before Mira Minkoba is the Hopebearer. Since t...
1480,373968,Fourteen Minutes : The Last Voyage of the Empr...,58,,,[],
1466,54007176,From the Stories of Old: A Collection of Fairy...,58,"fairy, worlds, sets, tales, mermaids, new worl...",,[],"In this international collection, new life is ..."
1476,26519134,IaED,58,"rema, darmik, rebel, will, loyalties tested, s...",,['566754'],Sentenced to be executed. Betrayed. Alone. Han...
1482,41700957,Protecting the Desert Princess,58,"layla, protect, protect but, can protect, gild...",,['668837'],Seven days without her crown! Princess Layla o...
1487,42942912,Renna Can't Remember,58,"village, princess, renna, now determined, disg...",,[],Free-spirited Princess Renna couldn't foresee ...
1479,26648933,Rowena's Key,58,"unlock, key, kingdom, prince, restorer, sorcer...",,['573540'],The key would unlock his future and the safety...


In [19]:
display(inspect_cluster(13))

,work_id,original_title,cluster_size,keywords_preview,tags_preview,series_preview,description_preview
1542,40816459,A Heart to Heal,58,"max, heather, gordon, simon, falls, cautious, ...",,['704432'],The Courage To Hope Guidance counselor Heather...
1550,7249360,Christmas at Bravo Ridge,58,"can live, matt, had, live, finally come, terms...",,['285328' '468451'],"Well, if lightning didn't strike twice, then h..."
1525,610745,"Colt's Choice (Bad in Boots, #3)",58,"colt, elise, ranch, rodeo, half, laid, has, hands",,['151048'],COLT'S CHOICE is novel length and is the third...
1521,16079600,Dating A Saint,58,"jim, lauren, married, anyway, learned, not, 42...",,['268966'],After divorcing her cheating ex-husband years ...
1519,41861288,Foundation of Trust,58,"owen, david, forgive, changes, ever, years, ch...",,['648153'],David Weller thought he had it all--a loving p...
1518,15142849,Hardening,58,"owen, daniel, thankfully, wonders if, wonders,...",,['243891'],"When Daniel leaves him, Owen wonders if he wil..."
1526,19008313,Her Heart for the Asking,58,"mandy, beau, rodeo, texas, uncle, gentry, had,...",,['502222'],Promises made...secrets kept... Mandy Morgan s...
1533,25072366,Intimate,58,"caleb, they ve, sex, ve, years has, even best,...",,['568801'],"They've loved each other for years, but they'v..."
1523,18465014,Long Road Home,58,"wyatt, olivia, cowboy, ranch, chance, last, bl...",,['341655'],What does it take to be a cowboy? Wyatt Locke ...
1520,46029616,"Lovers, Losers, and You",58,"owen, andrew, brother, brother never, find ano...",,['807940'],How can you lose one dream and still find anot...


In [20]:
display(inspect_cluster(14))

,work_id,original_title,cluster_size,keywords_preview,tags_preview,series_preview,description_preview
1601,6694929,A Highlander's Homecoming,57,"isabella, faerie, scotland, magic, robbie, hom...",,['171308'],When Robert MacQuarrie was swept forward in ti...
1609,26687508,A Scandal to Remember,57,"ship, charles, jane, burke, majesty, own name,...",,['563997'],"For too long, Miss Jane Burke's father has tak..."
1605,23576011,Blood of the King,57,"shaman, kingdom, king, curse, dying, can, born...",,['493148'],A kingdom torn by war. A curse whispered by dy...
1599,843971,Conquer The Night,57,"kyra, highland, family murdered, pawn, running...",,['685180'],His family murdered and his heart hardened by ...
1596,16388881,Darkest Highlander,57,"hearts, darkest, magic, highlander, own magic,...",,['266191' '839213'],"In the acclaimed Dark Sword series, the darkes..."
1575,1064605,Deirdre (The Fires of Gleannmara series #3),57,"saxon, princess, irish, god, king, no, instrum...",,['259346'],"Deirdre, princess of Gleannmara, is kidnapped ..."
1611,6679708,Highland Beast,57,"mirror, modern, highlander, lycan, yearned, ri...",,['260681'],From the moment she looked through the antique...
1582,28161405,"Highland Destiny (Highland Destiny, #4)",57,"highland, andi, druid, ian, destiny, magick, l...",,['592545'],"Highland Destiny: A tale of betrayal, murder, ..."
1606,24266348,Highland Master,57,"knight, entrancing, risks everything, life sti...",,['519642'],Lady Triona McKee's life is under desperate si...
1591,15719559,Highlander for the Holidays,57,"jessie, ian, imbued, finds more, hermit, highl...",,['261854'],"After a brutal attack, Jessie Pringle moved to..."


In [21]:
display(inspect_cluster(15))

,work_id,original_title,cluster_size,keywords_preview,tags_preview,series_preview,description_preview
1642,13737752,"Avengers Academy, Volume 1: Permanent Record",55,"hands, pym, hank, class but, earth mightiest, ...",,['319113' '868971'],"Determined to keep the tradition alive, foundi..."
1653,21491366,Captain America & Hawkeye,55,"hawkeye, captain america, cap, captain, americ...",,['978257' '926829'],Something's rotten in the San Andres Mountains...
1648,154942,Civil War: Fantastic Four,55,"fantastic four, fantastic, member, four, oppos...",,['910120' '1113510' '1069762'],One member of the Fantastic Four lies hospital...
1655,222130,"Essential X-Men, Vol. 7 (Marvel Essentials)",55,"mutant, recruiting, fantastic four, mobile, mo...",,['291328' '502848'],"In the wake of the Marauders' Mutant Massacre,..."
1656,315563,"Fantastic Four Visionaries: John Byrne, Vol. 7",55,"mightiest, avengers, annual, earth, earth migh...",,['287283' '400958' '1069729' '1113572'],Innovative 80s action The World's Greatest Sup...
1650,154947,Fantastic Four: First Family,55,,,['1069895'],
1667,49796428,"Mighty Thor, Vol. 2: Lords of Midgard",55,"enemy, evil, loki, caught middle, solomon, som...",,['974552'],The legend continues with Thors new and old! L...
1660,44751549,Miles Morales: Ultimate Spider-Man Ultimate Co...,55,"spider man, spider, miles, miles morales, mora...",,['831082' '1137983'],Miles Morales is still getting used to being S...
1634,41636255,"Miles Morales: Ultimate Spider-Man, Volume 1: ...",55,"spider, miles, spider man, peter, miles morale...",,['664334'],"Miles Morales, the ultimate Spider-Man, is bac..."
1663,2732060,Secret Invasion: The Infiltration,55,"number, fantastic, new avengers, fantastic fou...",,['295711'],Presents a close encounter of the Skrull kind ...


In [22]:
display(inspect_cluster(16))

,work_id,original_title,cluster_size,keywords_preview,tags_preview,series_preview,description_preview
1688,27636663,Chasing Lady Midnight,43,"squad, superhero, midnight, super, keeping, no...",,['982142'],The Mighty Superhero Security Squad is suppose...
1705,248264,Incredible Hulk: Tempest Fugit,43,,,['1081575' '632286' '413612' '737877'],
1692,6876265,Irredeemable Vol 2,43,"world greatest, superhero, super, greatest, wh...",,['187089'],Writer Mark Waid takes superhero comics to the...
1689,13539631,Irredemable Vol. 5,43,"world greatest, superhero, greatest, if world,...",,['187092'],Legendary comic writer Mark (KINGDOM COME) Wai...
1695,21740316,Past Forward-A Serial Novel: Volume I,43,"willow, finley, first, episodes, life, forward...",,['456689'],(Contains Episodes 1-5) Alone without friends ...
1723,929271,Super Friends!: Your Favorite Television Super...,43,"now returned, book form, 70s, wonder woman, br...",,['185801' '185814' '185827'],"In the mid-'70s, a team of artists re-transfor..."
1702,55830544,Supergirl Vol. 2,43,"batgirl, superman, wild west, tomorrow, emeral...",,['1071324'],The adventures of the Girl of Tomorrow continu...
1700,24670961,"Superman, Volume 3: Fury at World's End",43,"planet, earth, orion, comes, wonder woman, pas...",,['500452' '1122408'],H'El has come to Earth! When a mysterious ghos...
1718,46108699,Superman: American Alien,43,"superman, issue, american, eisner award, kisse...",,['904786'],Screenwriter and Eisner Award nominee Max Land...
1722,5846743,Superman: Shadows Linger (Superman (Graphic No...,43,"kurt, superman, lana, all star, kent, school s...",,['479643'],The nefarious Insect Queen debuts and not only...


In [23]:
display(inspect_cluster(17))

,work_id,original_title,cluster_size,keywords_preview,tags_preview,series_preview,description_preview
1745,2408,Cause of Death,40,"eddings, scarpetta, call someone, death police...",,['144429'],"It is New Year's Eve, the last day of Virginia..."
1748,3933994,"Death and the Author: How D.H. Lawrence Died, ...",40,"death, lawrence, material, dying, famous, even...",,[],At the heart of Death and the Author is a dram...
1756,6856114,Decay Inevitable,40,"territories, very end, secrets, death itself, ...",,[],DECAY INEVITABLE is a novel charting the grim ...
1742,42414324,Goin' Extinct: Tales From the Edge of Oblivion,40,"ways, could end, tug, more different, end, ent...",,[],Who can come up with a dozen or more different...
1752,45070849,Grief Is the Thing with Feathers,40,"part, grief, emptiness, extraordinary debut, s...",,[],"In a London flat, two young boys face the unbe..."
1731,26167869,"Love and Decay, Episode Six",40,"zombies, hendrix, tyler, miller, every, places...",,['720819' '557372'],Zombies- as if that wasn't enough to ruin any ...
1740,6222129,Soul Survivor (Book 3),40,,,[],
1741,18151205,Soul Survivors Hometown Tales Vol. 1,40,"horrors, these, survive, face, one only, end, ...",,[],Everyone you know is dead or dying. Your world...
1747,26869773,The End is Now,40,"apocalypse, apocalypse end, end, end now, trip...",,['565252'],Famine. Death. War. Pestilence. These are the ...
1730,13759245,The Living Dead Boy,40,"josh, zombie, zombies, hunters, them all, all,...",,['973551'],Josh Rondell is twelve years old and known as ...


In [24]:
display(inspect_cluster(18))

,work_id,original_title,cluster_size,keywords_preview,tags_preview,series_preview,description_preview
1774,25699856,Bloodlust,38,"win, ever, beaches, live happily, capabilities...",,['535455'],"Life's no fun being a dragon, especially when ..."
1792,49309376,Detective for the Dragon,38,"jared, nolan, rock, rock star, dragon, cheatin...",,['895593'],What happens when a rock star dragon hires a d...
1773,21457174,Dragon Haven,38,"dragons, rain wild, rain, kelsingra, their, th...",,['399458' '399459'],Centuries had passed since dragons last roamed...
1791,42465955,Dragon's Bride,38,"ewan, shannon, ryan, martin, dragon, fey, will...",,['673317'],"Book four in the Dragon and the Scholar Saga, ..."
1772,53000072,Dragons Like it Hot,38,"dragon, shifter, clan, curvy, mate, their, pri...",,['1083661' '1093289'],11 scorching-hot dragon-shifter romance storie...
1770,16319429,Dragonswood,38,"tess, wilde, fairies, dragons, draws, witch, h...",,['489896'],"On Wilde Island, there is no peace between dra..."
1790,40728699,"Haunted By Her Dragon (Dragon Guards, #3)",38,"not, fear, will not, never, universe, protects...",,['607294'],Life has never been easy for Dr. Samantha Malo...
1786,3428366,Magnifi-Cat,38,,,[],
1785,287916,"Meow is for Murder (Kendra Ballantyne, Petsitt...",38,"amanda, pet, ex, kendra, claws, ex wife, only ...",,['191480'],Another purr-fect Pet Sitting mystery. The las...
1780,44216022,On Cats,38,"bukowski, cat, animals, admired, cats, conside...",,[],A raw and tenderly funny look at the human-cat...


In [25]:
display(inspect_cluster(19))

,work_id,original_title,cluster_size,keywords_preview,tags_preview,series_preview,description_preview
1832,955407,Diamonds and Daisies,34,"potato, meat, virgin, nun, but own, romances, ...",,[],Novelist Sunny Parkinson might write scorching...
1812,6248661,Man of Mystery,34,,,[],
1819,26416112,Passion (Lost Love Series),34,"part time, time, part, will tell, furious when...",,['550063'],Cara is furious when her company passes her ov...
1808,14651123,Pure,34,"gods, alex, covenants, seth, some, threat, ale...",,['212023'],There is need. And then there is Fate... Being...
1830,7165256,Taken,34,"submit, some readers, never has, sex, won, rea...",,['189155'],"Diana is infuriated, humiliated. insanely arou..."
1815,19390590,The Bull Rider's Brother,34,"lizzie, james, weekend, problems, career, life...",,['477705'],Rodeo weekend is the start of the summer for t...
1811,19039,The Mysteries Collection (The Mystery of the S...,34,,,[],
1817,22021348,The Widow's Guide to Sex and Dating,34,"charlie, dates, claire, actor, dating, had, se...",,[],The Widow's Guide to Sex and Dating is Carole ...
1816,51824376,The Woman Who Lost Her Mojo,34,"charlie, list, jake, life, single, can, mojo, ...",,[],It's time for Charlie to ditch the life that's...
1809,26484337,NaN,34,,,,


In [26]:
display(inspect_cluster(20))

,work_id,original_title,cluster_size,keywords_preview,tags_preview,series_preview,description_preview
1864,226769,Awakening into Oneness,33,"oneness, blessing, awakening, experienced, tea...",,[],A fascinating discovery in southern India has ...
1843,1125324,Faith and Fire,33,,,['328391' '222161'],
1872,6841883,Fire Ice,33,"proclaimed, hair raising, opposition, distract...",,['172163'],"In the heart of the old Soviet Union, a mining..."
1842,1057710,Grace [Eventually]: Thoughts on Faith,33,,,[],
1859,202841,In the Buddha's Words: An Anthology of Discour...,33,"buddha, framework, teachings, works, translato...",,[],"The works of the Buddha can feel vast, and it ..."
1851,1245067,Introducing Eastern Philosophy,33,"eastern, philosophy, form, thought, westerners...",,['977613'],Eastern philosophy is the most ancient form of...
1856,19061044,Let it Go: Forgive So You Can Be Forgiven,33,"jakes, forgiveness, idea, let go, spiritual, l...",,[],"T.D. Jakes, New York Times bestselling author ..."
1860,2034298,Reinventing the Sacred,33,,,[],
1849,3837312,Return to The Sacred: Ancient Pathways to Spir...,33,"spiritual, yourself, will help, return, growth...",,[],Are you looking for inner peace? Do you seek a...
1850,231418,The Complete Idiot's Guide to Understanding Bu...,33,"buddhism, books, everyday, one, edition, ly, z...",,[],"Nirvana, Now or Never ! Two books in one: livi..."
